In [1]:
from env import LocalVol
import gymnasium as gym
import stable_baselines3
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.callbacks import EvalCallback
from loggers import TensorboardCallback
import numpy as np
import random
import os
import matplotlib.pyplot as plt

In [20]:
N = 10
Dynamics  = 'BS'
star_time = 0
T = 1
dT = 1/252
r = 0
mu = [0.15]
sigma = [0.5]
P = [[1]]

LVol = LocalVol(Dynamics = Dynamics, T = T, dT = dT, mu = mu, sigma = sigma, P = P)
LVol.seed(seed=random.seed(10))

env = gym.wrappers.TimeLimit(LVol, max_episode_steps=T)
env = Monitor(env, allow_early_resets=True)

In [21]:

print(env.action_space,env.observation_space)

Discrete(252, start=1) Discrete(1)


In [22]:
print(env.action_space.sample())

23


In [24]:
Nepisodes = 100
rew = []
act = []
tradingtimes = []

for i in range(Nepisodes):
    obs = env.reset()
    # obs = [[obs[0][i] for i in range(len(obs[0]))]]
    cont = True
    i = 0
    act.append([])
    tradingtimes.append([])
    reward_episode = 0
    while cont:
        action = 1#env.action_space.sample()
        obs, reward, done, info = LVol.step(action)
        act[-1].append(action)
        reward_episode += reward
        i += 1
        if done:
            cont = False
            tradingtimes[-1].append(env.unwrapped.tradingtimes)
            rew.append(reward_episode)


print(np.mean(rew),np.std(rew))

0.000737840773072201 8.829212040632842e-05


In [13]:
for i in range(10):
    print(i,tradingtimes[i],act[i] )

0 [[252]] [252]
1 [[252]] [252]
2 [[252]] [252]
3 [[252]] [252]
4 [[252]] [252]
5 [[252]] [252]
6 [[252]] [252]
7 [[252]] [252]
8 [[252]] [252]
9 [[252]] [252]


In [14]:
steps = 10000
model = PPO('MlpPolicy', DummyVecEnv([lambda: env]), learning_rate=0.001, verbose=1)
#model = PPO('MlpPolicy', LVol, learning_rate=0.001, verbose=1)
model.learn(total_timesteps=steps)

Using cpu device


AssertionError: The algorithm only supports (<class 'gymnasium.spaces.box.Box'>, <class 'gymnasium.spaces.discrete.Discrete'>, <class 'gymnasium.spaces.multi_discrete.MultiDiscrete'>, <class 'gymnasium.spaces.multi_binary.MultiBinary'>) as action spaces but Discrete(252, start=1) was provided